In [1]:
sentence = " 나는 밥을 먹었다 "

#문장 시작을 알리는 <start> 
source_sentence = "<start>" + sentence
#문장 마지막을 알리는 <end>
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


In [1]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


### 문장만 출력하고 싶을 때 
- 화자가 표기된 문장(0,3,6), 공백인 문장(2,5,9)를 제외시켜야 함
- 화자가 표기된 문장은 문장 끝이 : 로 끝남. 
- 공백인 문장은 길이가 0.
- 문장 끝이 : 로 끝나거나 문장 길이가 0이면 제외시키는 것

### 토큰화
- 텍스트 생성 모델에도 단어 사전을 만들게 됨.
- 문장을 일정한 기준으로 쪼개야 함.
- 이 과정을 토큰화라고 함.
- 가장 간단한 방법은 띄어쓰기를 기준으로 나누기.

# 데이터 전처리 과정

In [3]:
# 입력된 문장에서
#     1. 소문자로 바꾸고, 양쪽 공백을 지움
#     2. 특수문자 양쪽에 공백을 넣기
#     3. 여러개의 공백은 하나의 공백으로 바꿈
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈
#     5. 다시 양쪽 공백을 지움
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가

# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있음!

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


### 소스 문장 
- 자연어 처리 분야에서 모델의 입력이 되는 문장

### 타겟 문장
- 정답 역할을 하게 될 모델의 출력 문장   

### 정제 데이터 구축
- 정제 함수를 통해 만든 데이터셋에서 토큰화를 진행 후 끝 단어 < end >를 없애면 소스 문장이 됨.
- 첫 단어 < start >를 없애면 타켓 문장이 됨

In [4]:
#데이터 준비

#정제된 문장을 넣을 빈 리스트
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 건너뜀
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    #정제 후 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

## 벡터화
- tf.keras.preprocessing.text.Tokenizer 패키지는 정제된 데이터를 토큰화하고, 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며, 데이터를 숫자로 변환까지 해줌.

### 텐서(tensor)
- 숫자로 변환된 데이터

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용

def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요(빈칸)
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f980ca1ef10>


In [6]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [7]:
#텐서 데이터는 모두 정수, tokenizer에 구축 된 단어 사전의 인덱스임.

#구축 된 단어 사전 출력
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [8]:
#생성된 텐서를 소스와 타켓으로 분리하여 모델이 학습할 수 있게 함
#텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워 넣은 것
#0은 사전에는 없지만 패딩문자 <pad>가 될 것

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [9]:
#데이터셋 객체 형성
#이전에는 model.fit으로 모델에 데이터셋 제공했음
#텐서플로우는 텐서로 생성된 데이터를 이용해 tf.data.Dataset객체를 생성하는 방법을 흔히 사용
#tf.data.Dataset객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의 기능을 제공

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

# 데이터셋 생성 과정 정리 (데이터 전처리)
- 정규표현식을 이용한 corpus 생성
- tf.keras.prprocessing.text.Tokenizer를 이용해 courpus를 텐서로 변환
- tf.data.Dataset.from_tensor_slices()를 이용해 courpus 텐서를 tf.data.Dataset 객체로 변환


# 모델 설계

In [10]:
#1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense
#Embedding 레이어 ; 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔 줌
#워드벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용됨

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

#embedding_size :  워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기
#여기서는 256으로 정함. 데이터가 너무 많으면 혼란을 야기하기 때문
embedding_size = 256

#LSTM hiddenstate의 차원수 = hidden_size : 모델에 얼마나 많은 일꾼을 둘 것인지
#일꾼들은 같은 데이터를 보고 각자의 생각을 가지게 됨.
#충분한 데이터가 주어지면 올바른 결정을 내림.
#여기서는 1024로 정함. 역시 데이터가 너무 많으면 올바른 결정 못하기 때문
hidden_size = 1024

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~

#모델에 데이터를 아주 조금 태워보는 것
#모델이 아직 제대로 build되지 않았기 때문
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 1.71337640e-04, -2.74114398e-04,  7.86561600e-07, ...,
          3.66533413e-06,  4.08343942e-04, -1.62882294e-04],
        [ 9.39842430e-05, -8.04304655e-05, -4.62705357e-05, ...,
          4.35437367e-04,  6.65245985e-04, -3.00455460e-04],
        [ 1.54409063e-04, -1.89665327e-04,  2.82444496e-04, ...,
          9.30823211e-04,  7.51157582e-04, -9.86828090e-05],
        ...,
        [ 5.11249877e-04, -3.89733585e-04, -3.62372794e-03, ...,
         -5.02393988e-04,  7.86205637e-04,  5.86953771e-04],
        [ 5.94834681e-04, -4.79911483e-04, -3.75270797e-03, ...,
         -4.09254280e-04,  7.44833029e-04,  5.19899768e-04],
        [ 6.55893935e-04, -5.70120930e-04, -3.85342096e-03, ...,
         -3.22057167e-04,  6.99475990e-04,  4.47636558e-04]],

       [[ 1.71337640e-04, -2.74114398e-04,  7.86561600e-07, ...,
          3.66533413e-06,  4.08343942e-04, -1.62882294e-04],
        [ 2.43550967e-04, -4.49213141e-04, -1.

- 출력된 shape를 보면 (256,20,7001)임
- 7001은 Dense 레이어의 출력 차원 수
- 7001개 단어 중 어느 단어의 확률이 가장 높은지 모델링 해야 하기 대문
- 256은 배치 사이즈, dataset.take(1)을 통해 1개의 배치, 256개의 문장 데이터를 가져온 것
- 20 : tf.keras.layers.LSTM(hidden_size, return_sequences=True)로 호출한 LSTM 레이어에서 return_sequences=True 라고 지정한 부분 때문에, 본인에게 입력된 시퀀스 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미가 되므로 20이 나오게 된 것.
- 만약 return_sequences=False 였으면 LSTM 레이어는 1개의 벡터만 출력할 것.


- 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모르는데 20은 어떻게?
  - 데이터를 입력받으면서 알게 된 것. 데이터셋의  max_len이 20으로 맞춰져 있음

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


- 모델이 입력 시퀀스의 길이를 모르기 대문에 Output Shape를 특정할 수 없음.
- 모델 파라미터 사이즈는 측정 됨. 22million 정도

# 모델 학습

In [13]:
# optimizer와 loss등은 차차 배웁니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 40s 399ms/step - loss: 4.3968
Epoch 2/30
93/93 [==============================] - 37s 397ms/step - loss: 2.8134
Epoch 3/30
93/93 [==============================] - 37s 399ms/step - loss: 2.7207
Epoch 4/30
93/93 [==============================] - 37s 402ms/step - loss: 2.6196
Epoch 5/30
93/93 [==============================] - 38s 403ms/step - loss: 2.5395
Epoch 6/30
93/93 [==============================] - 38s 405ms/step - loss: 2.4927
Epoch 7/30
93/93 [==============================] - 37s 403ms/step - loss: 2.4393
Epoch 8/30
93/93 [==============================] - 38s 404ms/step - loss: 2.3951
Epoch 9/30
93/93 [==============================] - 38s 404ms/step - loss: 2.3348
Epoch 10/30
93/93 [==============================] - 37s 403ms/step - loss: 2.2824
Epoch 11/30
93/93 [==============================] - 38s 403ms/step - loss: 2.2487
Epoch 12/30
93/93 [==============================] - 37s 403ms/step - loss: 2.1933
Epoch 13/30
9

# 모델 평가

In [14]:
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

- while문
  - 텍스트를 실제로 생성해야 하는 시점
  - 타겟문장과 소스 문장이 없음
  - 텍스트 생성 태스크를 위해 테스트 데이터셋을 따로 생성해야 함
  - generate_text() 함수에서 init_sentence를 인자로 받아 텐서로 만들고 있음. 디폴트로는 < start > 단어 하나만 받음

In [15]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a changeling and two of my <unk> <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> she")

'<start> she is not fourteen . <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> I")

'<start> i ll tell you what you shall arraign my lord , <end> '

# 프로젝트 : 멋진 작사가 만들기

## 데이터 읽어오기

In [1]:
import glob
import os
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

#여러개 텍스트 파일 읽어서 raw_corpus에 저장
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


## 데이터 전처리

In [22]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0 or len(sentence) > 15 : continue
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    

    if idx > 20: break  
        
    print(sentence)

I was petrified
I grew strong
I will survive
Yeah


In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [3]:
#정제된 문장을 넣을 빈 리스트
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 건너뜀
    if len(sentence) == 0 or len(sentence) > 15 : continue
    if sentence[-1] == ":": continue
    
    #정제 후 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

In [32]:
def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    #corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    
    #준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    #입력 데이터의 시퀀스 길이를 일정하게 맞춤
    #만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춤
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  
    #문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    4   98 ...    0    0    0]
 [   2    4 1076 ...    0    0    0]
 [   2    4  111 ...    0    0    0]
 ...
 [   2    4   21 ...    0    0    0]
 [   2    4   21 ...    0    0    0]
 [   2    4   21 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fd6483b5450>


In [33]:
#tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성

#마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]  

#tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2    4   98 1336    3    0    0    0    0    0    0    0    0    0]
[   4   98 1336    3    0    0    0    0    0    0    0    0    0    0]


In [34]:
print(tensor.shape)
print(src_input.shape)
print(tgt_input.shape)

(14120, 15)
(14120, 14)
(14120, 14)


## 평가 데이터셋 분리

In [36]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val =  train_test_split(tensor, 
                                                    tensor, 
                                                    test_size=0.2, 
                                                    random_state=7)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (11296, 15)
Target Train: (11296, 15)


## 모델 형성

In [37]:
lib_size = tokenizer.num_words
embedding_size = 256
hidden_size = 1024

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(lib_size, embedding_size))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.Dense(lib_size))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 256)         3072000   
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 1024)        5246976   
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 1024)        8392704   
_________________________________________________________________
dense_2 (Dense)              (None, None, 12000)       12300000  
Total params: 29,011,680
Trainable params: 29,011,680
Non-trainable params: 0
_________________________________________________________________


In [38]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
353/353 [==============================] - 30s 79ms/step - loss: 2.2471
Epoch 2/10
353/353 [==============================] - 28s 79ms/step - loss: 0.7757
Epoch 3/10
353/353 [==============================] - 28s 79ms/step - loss: 0.5726
Epoch 4/10
353/353 [==============================] - 28s 80ms/step - loss: 0.4405
Epoch 5/10
353/353 [==============================] - 28s 80ms/step - loss: 0.3387
Epoch 6/10
353/353 [==============================] - 28s 80ms/step - loss: 0.2658
Epoch 7/10
353/353 [==============================] - 28s 80ms/step - loss: 0.2092
Epoch 8/10
353/353 [==============================] - 28s 80ms/step - loss: 0.1640
Epoch 9/10
353/353 [==============================] - 28s 80ms/step - loss: 0.1328
Epoch 10/10
353/353 [==============================] - 28s 80ms/step - loss: 0.1075


## 모델 평가

In [39]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [44]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love love love love love love love love love love love love love love love love love love '

In [43]:
generate_text(model, tokenizer, init_sentence="<start> i am", max_len=20)

'<start> i am am am am am am am am am am am am am am am am am am '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> you are", max_len=20)

'<start> you are <end> '